<a href="https://colab.research.google.com/github/BhargaviKatta/End-to-End-Medical-Chatbot/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyttsx3

In [5]:
import re
import pandas as pd
import pyttsx3
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier,_tree
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import csv
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.naive_bayes import GaussianNB
training = pd.read_csv('/content/sample_data/Training.csv')
testing= pd.read_csv('/content/sample_data/Testing.csv')
cols= training.columns
cols= cols[:-1]
x = training[cols]
y = training['prognosis']
y1= y


reduced_data = training.groupby(training['prognosis']).max()

#mapping strings to numbers
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
testx    = testing[cols]
testy    = testing['prognosis']
testy    = le.transform(testy)




clf1  = DecisionTreeClassifier()
clf = clf1.fit(x_train,y_train)
# print(clf.score(x_train,y_train))
# print ("cross result========")
scores = cross_val_score(clf, x_test, y_test, cv=3)
# print (scores)
print (scores.mean())



model=SVC(kernel='linear')
model.fit(x_train,y_train)
print("SVM accuracy: ",end="")
print(model.score(x_test,y_test))


from sklearn.ensemble import RandomForestClassifier #puja
classifier = RandomForestClassifier(n_estimators = 200, criterion = 'entropy', random_state=300)
classifier.fit(x_train,y_train)
y_pred = classifier.predict(x_test)
print('Random Forest Accuracy: %.3f' % accuracy_score(y_test, y_pred))

standardScaler = StandardScaler()
X = standardScaler.fit_transform(x)
classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
print("Naive bayes Accuracy:",accuracy_score(y_test, y_pred))

severityDictionary=dict()
medicationList = dict()
precautionDictionary=dict()
mhistory=dict()
food=dict()

symptoms_dict = {}

for index, symptom in enumerate(x):
       symptoms_dict[symptom] = index
def calc_condition(exp,days):
    sum=0
    for item in exp:
         sum=sum+severityDictionary[item]
    if((sum*days)/(len(exp)+1)>13):
        print("You should take the consultation from doctor.As you are suffering with "+str(days))
    else:
        print("It might not be that bad but you should take precautions.")


def getMedication():
    global medicationList
    with open('/content/sample_data/symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _description={row[0]:row[1]}
            medicationList.update(_description)
def getMhistory():
    global mhistory
    with open('/content/sample_data/symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _history={row[0]:row[2]}
            mhistory.update(_history)



def getSeverityDict():
    global severityDictionary
    with open('/content/sample_data/Symptom_severity.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        try:
            for row in csv_reader:
                _diction={row[0]:int(row[1])}
                severityDictionary.update(_diction)
        except:
            pass


def getprecautionDict():
    global precautionDictionary
    with open('/content/sample_data/symptom_precaution.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _prec={row[0]:[row[1],row[2],row[3],row[4]]}
            precautionDictionary.update(_prec)
def getfood():
    global food
    with open('/content/sample_data/symptom_precaution.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            line_count = 0
            _precf={row[0]:row[5]}
            food.update(_precf)


def getInfo():
    print("-----------------------------------HealthCare ChatBot-----------------------------------")
    print("Hey. Thís is Svasthya your Helper. I am here to assist regarding your health and wellbeing 24/7.")

def check_pattern(dis_list,inp):
    pred_list=[]
    inp=inp.replace(' ','_')
    patt = f"{inp}"
    regexp = re.compile(patt)
    pred_list=[item for item in dis_list if regexp.search(item)]
    if(len(pred_list)>0):
        return 1,pred_list
    else:
        return 0,[]
def sec_predict(symptoms_exp):
    df = pd.read_csv('/content/sample_data/Training.csv')
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    rf_clf = DecisionTreeClassifier()
    rf_clf.fit(X_train, y_train)

    symptoms_dict = {symptom: index for index, symptom in enumerate(X)}
    input_vector = np.zeros(len(symptoms_dict))
    for item in symptoms_exp:
      input_vector[[symptoms_dict[item]]] = 1

    return rf_clf.predict([input_vector])


def print_disease(node):
    node = node[0]
    val  = node.nonzero()
    disease = le.inverse_transform(val[0])
    return list(map(lambda x:x.strip(),list(disease)))

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    chk_dis=",".join(feature_names).split(",")
    symptoms_present = []

    while True:

        print("\nEnter the symptom you are experiencing  \t\t",end="->")
        disease_input = input("")
        conf,cnf_dis=check_pattern(chk_dis,disease_input)
        if conf==1:
            print("searches related to input: ")
            for num,it in enumerate(cnf_dis):
                print(num,")",it)
            if num!=0:
                print(f"Select the one you meant (0 - {num}):  ", end="")
                conf_inp = int(input(""))
            else:
                conf_inp=0

            disease_input=cnf_dis[conf_inp]
            break
        else:
            print("Enter valid symptom.")

    while True:
        try:
            num_days=int(input("Okay. From how many days ? : "))
            break
        except:
            print("Enter valid input.")

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]

            if name == disease_input:
                val = 1
            else:
                val = 0
            if  val <= threshold:
                recurse(tree_.children_left[node], depth + 1)
            else:
                symptoms_present.append(name)
                recurse(tree_.children_right[node], depth + 1)
        else:
            present_disease = print_disease(tree_.value[node])

            red_cols = reduced_data.columns
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]

            print("Are you experiencing any ")
            symptoms_exp=[]
            for syms in list(symptoms_given):
                inp=""
                print(syms,"? : ",end='')
                while True:
                    inp=input("")
                    if(inp=="yes" or inp=="no"):
                        break
                    else:
                        print("provide proper answers i.e. (yes/no) : ",end="")
                if(inp=="yes"):
                    symptoms_exp.append(syms)

            second_prediction=sec_predict(symptoms_exp)

            calc_condition(symptoms_exp,num_days)
            if(present_disease[0]==second_prediction[0]):

                print("Please take following medication:",medicationList[present_disease[0]])
                print("The following medical condition people should not use this medication",mhistory[present_disease[0]])

            else:

                print("We are unable to catch your medical condition,Please consult the doctor.yes")


            precution_list=precautionDictionary[present_disease[0]]
            print("Take following measures : ")
            for  i,j in enumerate(precution_list):
                print(i+1,")",j)
            food_list=food[present_disease[0]]
            print("Please take following the food diet : ",food_list)

    recurse(0, 1)
getSeverityDict()
getMedication()
getMhistory()
getprecautionDict()
getInfo()
getfood()
tree_to_code(clf,cols)
print("----------------------------------------------------------------------------------------")

0.9716778868343212
SVM accuracy: 1.0
Random Forest Accuracy: 1.000
Naive bayes Accuracy: 1.0
-----------------------------------HealthCare ChatBot-----------------------------------
Hey. Thís is Svasthya your Helper. I am here to assist regarding your health and wellbeing 24/7.

Enter the symptom you are experiencing  		->cold
searches related to input: 
0 ) cold_hands_and_feets
Okay. From how many days ? : 3
Are you experiencing any 
vomiting ? : no
yellowish_skin ? : no
abdominal_pain ? : no
swelling_of_stomach ? : no
distention_of_abdomen ? : yes
history_of_alcohol_consumption ? : no
fluid_overload.1 ? : no
It might not be that bad but you should take precautions.
We are unable to catch your medical condition,Please consult the doctor.yes
Take following measures : 
1 ) stop alcohol consumption
2 ) consult doctor
3 ) medication
4 ) follow up
Please take following the food diet :  Stop drinking alcohol entirely,Maintain a Balanced Diet,Limt Sodium intake,Include sources of healthy fa

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
